# Convolutional neural network (version 2)
- 미리 만들어진 모델을 불러와서 학습을 해봅시다.
- nets/kthvgg_slim.py 참고

In [1]:
import random
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from nets.kthvgg_slim import vgg

## 0. Load MNIST data
- 이번에는 one_hot=False로 데이터를 불러와봅시다.

In [2]:
# one_hot=False
mnist = input_data.read_data_sets("MNIST_idx3/", one_hot=False)

Extracting MNIST_idx3/train-images-idx3-ubyte.gz
Extracting MNIST_idx3/train-labels-idx1-ubyte.gz
Extracting MNIST_idx3/t10k-images-idx3-ubyte.gz
Extracting MNIST_idx3/t10k-labels-idx1-ubyte.gz


In [3]:
X_trn, Y_trn = mnist.train.images, mnist.train.labels
X_val, Y_val = mnist.validation.images, mnist.validation.labels
X_test, Y_test = mnist.test.images, mnist.test.labels

In [4]:
print(Y_trn[0:5])

[7 3 4 6 1]


In [5]:
num_trn = Y_trn.shape[0]
num_val = Y_val.shape[0]
num_test = Y_test.shape[0]

print("Number of training points: ", num_trn)
print("Number of validation points: ", num_val)
print("Number of test points: ", num_test)

Number of training points:  55000
Number of validation points:  5000
Number of test points:  10000


In [6]:
dim_X = X_trn.shape[1]
pixel_X = int(np.sqrt(dim_X)) # np.sqrt의 출력이 float32이므로, 이를 int 자료형으로 변경
# dim_Y = Y_trn.shape[1]

print("Dimension of X: %d (%d x %d)" % (dim_X, pixel_X, pixel_X))
# print("Dimension of Y: ", dim_Y)
print("Dimension of Y: None.. Y is a array of integers.")

Dimension of X: 784 (28 x 28)
Dimension of Y: None.. Y is a array of integers.


In [7]:
X_trn = X_trn.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [8]:
X_trn.shape

(55000, 28, 28, 1)

## 1. Build the graph
Tensorflow에서는 모델을 'graph'로 구현한다.

### 1.1. Placeholder for inputs and outputs
- Shape of the placeholder for inputs: [batch_size, input_dimension]
- Shape of the placeholder for outputs: [batch_size]
- Placeholder의 batch_size를 None으로 하면, placeholder에 들어가기 전에 batch size를 조절해야 함

In [9]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1], name="Inputs")

In [10]:
Y = tf.placeholder(tf.int32, [None], name="Labels")

In [11]:
test = X.get_shape()
# test = X.get_shape()[1]
print(test)
# print(type(test))

(?, 28, 28, 1)


### 1.2. Build the model

- Weight와 bias, 그래프 구조를 생성하는 함수를 만들어봅시다.
- 여기에서는 2개의 hidden layers를 생성하겠습니다.

In [12]:
# def fully_connected(inputs, hidden_dim_1, hidden_dim_2, num_classes, scope='SimpleFCN'):
#     """
#     [fully_connected] 2개의 hidden layer를 갖는 feed-forward network 생성
    
#     [Args]
#       - inputs: 입력 데이터를 위한 placeholder
#       - hidden_dim_1: 첫 번째 은닉층의 노드 수
#       - hidden_dim_2: 두 번째 은닉층의 노드 수
#       - num_classes: 예측하고자 하는 클래스의 수 (= 출력층의 노드 수))
#       - Scope: default value ("SimpleFCN")
#     """
#     # Inputs에서 1차원의 텐서들이 placeholder로 들어온다고 가정
#     input_dim = inputs.get_shape()[1]
    
#     # tf.truncated_normal_initializer의 형태를 간소화
#     trunc_normal = lambda stddev: tf.truncated_normal_initializer(mean=0.0, stddev=stddev)
    
#     # tf.constant_initializer의 형태를 간소화
#     constant = lambda value: tf.constant_initializer(value=value)
    
#     # Define "end_points"
#     end_points = {}
    
#     with tf.variable_scope(scope):
#         with tf.variable_scope('HiddenLayer_1'):
#             W_h1 = tf.get_variable("weights", [input_dim, hidden_dim_1], initializer=trunc_normal(0.1))
#             b_h1 = tf.get_variable("biases", [hidden_dim_1], initializer=constant(0.0))
#             h1 = tf.nn.relu(tf.matmul(inputs, W_h1)+ b_h1, name="Activation")
#             end_points['h1'] = h1
            
#         with tf.variable_scope('hiddenLayer_2'):
#             W_h2 = tf.get_variable("weights", [hidden_dim_1, hidden_dim_2], initializer=trunc_normal(0.09))
#             b_h2 = tf.get_variable("biases", [hidden_dim_2], initializer=constant(0.01))
#             h2 = tf.nn.relu(tf.matmul(h1, W_h2) + b_h2, name="Activation")
#             end_points['h2'] = h2
            
#         with tf.variable_scope('OutputLayer'):
#             W_o = tf.get_variable("weights", [hidden_dim_2, num_classes], initializer=trunc_normal(0.1))
#             b_o = tf.get_variable("biases", [num_classes], initializer=constant(0.0))
#             with tf.variable_scope('Logits'): logits = tf.matmul(h2, W_o) + b_o
# #             logits = tf.matmul(h2, W_o) + b_o
#             end_points['logits'] = logits
    
#     return logits, end_points
    

In [14]:
logits, end_points = vgg(inputs=X, num_classes=10)

In [15]:
print(logits)

Tensor("KTH_VGG/Logits/BiasAdd:0", shape=(?, 10), dtype=float32)


In [16]:
# Print my end points
pprint(end_points)

OrderedDict([('KTH_VGG/Conv1',
              <tf.Tensor 'KTH_VGG/Conv1/Relu:0' shape=(?, 28, 28, 32) dtype=float32>),
             ('KTH_VGG/Pool1',
              <tf.Tensor 'KTH_VGG/Pool1/MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>),
             ('KTH_VGG/Conv2',
              <tf.Tensor 'KTH_VGG/Conv2/Relu:0' shape=(?, 14, 14, 64) dtype=float32>),
             ('KTH_VGG/Pool2',
              <tf.Tensor 'KTH_VGG/Pool2/MaxPool:0' shape=(?, 7, 7, 64) dtype=float32>),
             ('KTH_VGG/Conv3',
              <tf.Tensor 'KTH_VGG/Conv3/Relu:0' shape=(?, 7, 7, 128) dtype=float32>),
             ('KTH_VGG/Pool3',
              <tf.Tensor 'KTH_VGG/Pool3/MaxPool:0' shape=(?, 3, 3, 128) dtype=float32>),
             ('KTH_VGG/Conv4',
              <tf.Tensor 'KTH_VGG/Conv4/Relu:0' shape=(?, 3, 3, 256) dtype=float32>),
             ('KTH_VGG/Pool4',
              <tf.Tensor 'KTH_VGG/Pool4/MaxPool:0' shape=(?, 1, 1, 256) dtype=float32>),
             ('KTH_VGG/Flatten5',
              <t

## 2. Training

### 2.1. Loss function
- Classification 문제에서 제일 많이 사용하는 loss function은 **cross-entropy**

두 가지 옵션이 있음.
- tf.nn.softmax_cross_entropy_with_logits: Y가 one-hot encoded 되어 있을 때
- tf.nn.sparse_softmax_cross_entropy_with_logits: Y가 class에 대한 index 값일 때)

In [17]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=Y))

### 2.2. Training operator
- First, define the oprimizer. (**optimizer**)
- And then, define training operator. (**train_op**)

자주 사용하는 optimizer로는 다음과 같은 것들이 있음.
- tf.train.GradientDescentOptimizer
- tf.train.AdagradOptimizer
- tf.train.MomentumOptimizer
- **tf.train.AdamOptimizer** (많은 연구자들이 사용)

자세한 사항은 [TensorFlow API_guides: Training](https://www.tensorflow.org/api_guides/python/train) 참조!

In [18]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

In [19]:
train_op = optimizer.minimize(cost)

### 2.3. Predicting operator
- correct_prediction: boolean (True or False)
- accuracy: 먼저 correct_prediction을 float32로 변환 후에 배치 내 평균을 계산

**tf.nn.in_top_k(x, y, k)**
- tf.nn.in_top_k(x, y, k)는 prediction x의 상위 k개의 결과가 true label y를 포함하는지를 계산
- 이에 대한 output은 boolean 으로 나오므로, 이를 0, 1로 바꿔주기 위해서 tf.cast를 이용하여 float32로 변환한 이후에 accuracy를 계산한다.

In [20]:
correct_prediction = tf.nn.in_top_k(logits, Y, 1)

In [21]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [22]:
print(correct_prediction)
print(accuracy)

Tensor("InTopK:0", shape=(?,), dtype=bool)
Tensor("Mean_1:0", shape=(), dtype=float32)


### 2.4. Initializer

In [23]:
init = tf.global_variables_initializer()

### 2.5. Run the session
- 앞서 만든 graph, operator 등을 돌리는 과정

In [24]:
NUM_EPOCHS = 30
BATCH_SIZE = 128

In [25]:
# Batch 인덱스 생성
start_idx = range(0, num_trn, BATCH_SIZE)
end_idx = range(BATCH_SIZE, num_trn + 1, BATCH_SIZE)
for start, end in zip(start_idx, end_idx): print(start, '\t', end)

0 	 128
128 	 256
256 	 384
384 	 512
512 	 640
640 	 768
768 	 896
896 	 1024
1024 	 1152
1152 	 1280
1280 	 1408
1408 	 1536
1536 	 1664
1664 	 1792
1792 	 1920
1920 	 2048
2048 	 2176
2176 	 2304
2304 	 2432
2432 	 2560
2560 	 2688
2688 	 2816
2816 	 2944
2944 	 3072
3072 	 3200
3200 	 3328
3328 	 3456
3456 	 3584
3584 	 3712
3712 	 3840
3840 	 3968
3968 	 4096
4096 	 4224
4224 	 4352
4352 	 4480
4480 	 4608
4608 	 4736
4736 	 4864
4864 	 4992
4992 	 5120
5120 	 5248
5248 	 5376
5376 	 5504
5504 	 5632
5632 	 5760
5760 	 5888
5888 	 6016
6016 	 6144
6144 	 6272
6272 	 6400
6400 	 6528
6528 	 6656
6656 	 6784
6784 	 6912
6912 	 7040
7040 	 7168
7168 	 7296
7296 	 7424
7424 	 7552
7552 	 7680
7680 	 7808
7808 	 7936
7936 	 8064
8064 	 8192
8192 	 8320
8320 	 8448
8448 	 8576
8576 	 8704
8704 	 8832
8832 	 8960
8960 	 9088
9088 	 9216
9216 	 9344
9344 	 9472
9472 	 9600
9600 	 9728
9728 	 9856
9856 	 9984
9984 	 10112
10112 	 10240
10240 	 10368
10368 	 10496
10496 	 10624
10624 	 1075

In [26]:
# Batch의 개수
print(len(start_idx))

430


In [ ]:
trn_cost_list = list()
val_cost_list = list()
val_accuracy_list = list()

In [ ]:
with tf.Session() as sess, tf.device("/cpu:0"):
    # Variable initialization
    sess.run(init)
    
    # Indices for constructing batches
    start_idx = range(0, num_trn, BATCH_SIZE)
    end_idx = range(BATCH_SIZE, num_trn + 1, BATCH_SIZE)
    
    NUM_BATCHES = len(start_idx)
    
    for epoch in range(0,NUM_EPOCHS):

        # Set "trn_cost" as 0 before starting the epoch
        trn_cost = 0
        
        # Training phase
        for start, end in zip(start_idx, end_idx):

            # Construct the input batch
            batch_xs = X_trn[start:end]
            batch_ys = Y_trn[start:end]
            
            # Calculate cost
            tmp_cost, _ = sess.run([cost, train_op], feed_dict={X: batch_xs, Y: batch_ys})
            trn_cost += tmp_cost
        
        trn_cost = trn_cost / NUM_BATCHES
        trn_cost_list.append(trn_cost)
        print("[{} epoch] training cost {:0.4f}".format((epoch + 1), trn_cost))
        
        # Validation phase
        if (epoch + 1) % 10 == 0:
            val_cost, val_accuracy = sess.run([cost, accuracy], feed_dict={X: X_val, Y: Y_val})
            val_cost_list.append(val_cost)
            val_accuracy_list.append(val_accuracy)
            print("\t[{} epoch] validation accuracy {:0.4f}".format((epoch + 1), val_accuracy))
            
    # Test phase
    test_accuracy = sess.run(accuracy, feed_dict={X: X_test, Y: Y_test})
    print("\n")
    print("Test accuracy: {:0.4f}".format(test_accuracy))

[1 epoch] training cost 2.2990
[2 epoch] training cost 2.2963
[3 epoch] training cost 2.2963
[4 epoch] training cost 2.2963
[5 epoch] training cost 2.2963
[6 epoch] training cost 2.2963
[7 epoch] training cost 2.2963
